# FEM-BEM coupling of Poisson-Boltzmann 

In [1]:
import numpy as np
from mpi4py import MPI
from dolfinx.fem import FunctionSpace
from dolfinx.mesh import create_unit_square, create_unit_cube
from dolfinx.geometry import (compute_colliding_cells,
                              compute_collisions_points)

def get_local_coordinates(vertices, point):
    """Get the local coordinates of the point in the cell."""
    origin = vertices[0]
    axes = [v - origin for v in vertices[1:]]
    tdim = 3
    if len(axes) == 2:
        axes.append(np.cross(axes[0], axes[1]))
        tdim = 2

    assert len(axes) == 3


    return np.linalg.solve(np.array(axes).T, point - origin)[:tdim]


def assemble_point_sources(space, points, weights):
    vector = np.zeros(space.dofmap.index_map.size_global)

    mesh = space.mesh
    for point, weight in zip(points, weights):
        # Get cell
        tree = dolfinx.geometry.bb_tree(mesh, mesh.geometry.dim)
        cell_candidates = compute_collisions_points(tree, point)
        # This gets the first cell. Would it be better to put a fraction of the delta function into each cell instead?
        cell = compute_colliding_cells(mesh, cell_candidates, point)[0]

        # Get local coordinates
        # Note: this currently only works for affine triangles and tetrahedra
        v = [mesh.geometry.x[i] for i in mesh.geometry.dofmap[cell]]
        local_coordinates = get_local_coordinates(v, point)

        # Note: this currently only works for scalar-valued elements that use an identity push forward map
        values = space.element.basix_element.tabulate(0, [local_coordinates])[0, 0, :, 0]
        dofs = space.dofmap.cell_dofs(cell)

        for d, v in zip(dofs, values):
            vector[d] += v * weight

    return vector


def test_point_sources_triangle():
    points = np.array([(1/4, 1/6, 0), (1/10, 6/10, 0)])

    mesh = create_unit_square(MPI.COMM_WORLD, 2, 2)
    space = FunctionSpace(mesh, ("Lagrange", 1))

    vector = assemble_point_sources(space, points, [1, 1])

    a = [i for i in vector]
    a.sort()
    assert np.isclose(a[-1], 4/5)
    assert np.isclose(a[-2], 1/2)
    assert np.isclose(a[-3], 1/3)
    assert np.isclose(a[-4], 1/5)
    assert np.isclose(a[-5], 1/6)
    assert np.isclose(a[-6], 0)
    assert np.isclose(a[0], 0)

    vector = assemble_point_sources(space, points, [2, 1])

    a = [i for i in vector]
    a.sort()
    assert np.isclose(a[-1], 1)
    assert np.isclose(a[-2], 4/5)
    assert np.isclose(a[-3], 2/3)
    assert np.isclose(a[-4], 1/3)
    assert np.isclose(a[-5], 1/5)
    assert np.isclose(a[-6], 0)
    assert np.isclose(a[0], 0)


def test_point_sources_tetrahedron():
    points = np.array([(1/4, 1/6, 0), (1/2, 1/2, 15/16)])

    mesh = create_unit_cube(MPI.COMM_WORLD, 2, 2, 2)
    space = FunctionSpace(mesh, ("Lagrange", 1))

    vector = assemble_point_sources(space, points, [1, 1])

    a = [i for i in vector]
    a.sort()
    assert np.isclose(a[-1], 7/8)
    assert np.isclose(a[-2], 1/2)
    assert np.isclose(a[-3], 1/3)
    assert np.isclose(a[-4], 1/6)
    assert np.isclose(a[-5], 1/8)
    assert np.isclose(a[-6], 0)
    assert np.isclose(a[0], 0)


    mesh = create_unit_cube(MPI.COMM_WORLD, 2, 2, 2)
    space = FunctionSpace(mesh, ("Lagrange", 1))

    vector = assemble_point_sources(space, points, [2, 1])

    a = [i for i in vector]
    a.sort()
    assert np.isclose(a[-1], 1)
    assert np.isclose(a[-2], 7/8)
    assert np.isclose(a[-3], 2/3)
    assert np.isclose(a[-4], 1/3)
    assert np.isclose(a[-5], 1/8)
    assert np.isclose(a[-6], 0)
    assert np.isclose(a[0], 0)

## Solving problems for each of the mesh

In [2]:
from mpi4py import MPI 
from dolfinx.mesh import create_unit_cube
from scipy.sparse import diags

def poisson_boltzman_jn_var(mesh):
    
    nor = ufl.FacetNormal(mesh) 
    
    # Spaces for FEM and BEM #
    fenics_space = FunctionSpace(mesh, ("Lagrange", 1))
    trace_space, trace_matrix = \
    fenicsx.fenics_to_bempp_trace_data(fenics_space)
    bempp_space = bempp.api.function_space(trace_space.grid, "P", 1)

    # Mesh size information #
    fem_size = fenics_space.dofmap.index_map.size_global
    print("FEM dofs: {0}".format(fem_size))
    print("BEM dofs: {0}".format(bempp_space.global_dof_count))
    hmax = trace_space.grid.maximum_element_diameter
    
    epsilon_fem = dolfinx.fem.Function(fenics_space)
    epsilon_fem.interpolate(fun_exp)
    
    # Starting set-up time
    start = time.time()     
    
    # FEM discrete variables
    u = ufl.TrialFunction(fenics_space)
    v = ufl.TestFunction(fenics_space)
        
    # BEM operators
    id_op = bempp.api.operators.boundary.sparse.identity(trace_space, bempp_space, bempp_space)
    mass = bempp.api.operators.boundary.sparse.identity(bempp_space, bempp_space, trace_space)
    
    if fmm:
        dlp = bempp.api.operators.boundary.modified_helmholtz.double_layer(trace_space, bempp_space, bempp_space, w, assembler="fmm")
        slp = bempp.api.operators.boundary.modified_helmholtz.single_layer(bempp_space, bempp_space, bempp_space, w, assembler="fmm")
        dlp_0 = bempp.api.operators.boundary.laplace.double_layer(trace_space, bempp_space, bempp_space, assembler="fmm")
        slp_0 = bempp.api.operators.boundary.laplace.single_layer(bempp_space, bempp_space, bempp_space, assembler="fmm")
    else:
        dlp = bempp.api.operators.boundary.modified_helmholtz.double_layer(trace_space, bempp_space, bempp_space, w)
        slp = bempp.api.operators.boundary.modified_helmholtz.single_layer(bempp_space, bempp_space, bempp_space, w)
        dlp_0 = bempp.api.operators.boundary.laplace.double_layer(trace_space, bempp_space, bempp_space)
        slp_0 = bempp.api.operators.boundary.laplace.single_layer(bempp_space, bempp_space, bempp_space)
    
    ######################### Matrix and RHS ##################################
    
    # RHS for FEM
    rhs_fem = assemble_point_sources(fenics_space, PC, Q)
        
    # RHS for BEM
    rhs_bem = np.zeros(bempp_space.global_dof_count)
    # Global RHS
    rhs = np.concatenate([rhs_fem, rhs_bem])
    
    blocks = [[None,None],[None,None]]
    blocks_vac = [[None,None],[None,None]]
        
    # FEM matrices
    trace_op = LinearOperator(trace_matrix.shape, lambda x:trace_matrix*x)
    
    A = fenicsx.FenicsOperator(epsilon_fem*ufl.inner(ufl.nabla_grad(u),
                                     ufl.nabla_grad(v))  * ufl.dx)  
    A_vac = fenicsx.FenicsOperator(em*ufl.inner(ufl.nabla_grad(u),
                                     ufl.nabla_grad(v))  * ufl.dx)  
     
    # Blocked matrix
    blocks[0][0] = A.weak_form()
    blocks[0][1] = -em*trace_matrix.T @ ( mass.weak_form().to_sparse())
    blocks[1][0] = (.5 * id_op - dlp).weak_form() * trace_op
    blocks[1][1] = slp.weak_form()*(em/es)

    blocked = BlockedDiscreteOperator(np.array(blocks))
    
    # Vaccum case
    blocks_vac[0][0] = A_vac.weak_form()
    blocks_vac[0][1] = blocks[0][1]
    blocks_vac[1][0] = (.5 * id_op - dlp_0).weak_form() * trace_op
    blocks_vac[1][1] = slp_0.weak_form()

    blocked_vacuum = BlockedDiscreteOperator(np.array(blocks_vac))
    
    
    ######################### Preconditioner and solvers ##################################
    
    # Diagonal Scaling #
    P1 = diags(1./blocked[0,0].to_sparse().diagonal())
    PV = diags(1./blocked_vacuum[0,0].to_sparse().diagonal())

    # Mass matrix preconditioner #
    P2 = InverseSparseDiscreteBoundaryOperator(
        bempp.api.operators.boundary.sparse.identity(
            bempp_space, bempp_space, bempp_space).weak_form())

    # Create a block diagonal preconditioner object using the Scipy LinearOperator class
    def apply_prec(x):
        """Apply the block diagonal preconditioner"""
        m1 = P1.shape[0]
        m2 = P2.shape[0]
        n1 = P1.shape[1]
        n2 = P2.shape[1]

        res1 = P1.dot(x[:n1]) 
        res2 = P2.dot(x[n1:])
        return np.concatenate([res1, res2])

    p_shape = (P1.shape[0] + P2.shape[0], P1.shape[1] + P2.shape[1])
    P = LinearOperator(p_shape, apply_prec, dtype=np.dtype('float64'))

    def vac_apply_prec(x):
        """Apply the block diagonal preconditioner"""
        m1 = PV.shape[0]
        m2 = P2.shape[0]
        n1 = PV.shape[1]
        n2 = P2.shape[1]

        res1 = PV.dot(x[:n1]) 
        res2 = P2.dot(x[n1:])
        return np.concatenate([res1, res2])

    p_shape = (PV.shape[0] + P2.shape[0], PV.shape[1] + P2.shape[1])
    P_vac = LinearOperator(p_shape, vac_apply_prec, dtype=np.dtype('float64'))
     
    end = time.time()
    set_time = (end - start) 

    from scipy.sparse.linalg import gmres
    start = time.time()     
    soln, info = gmres(blocked, rhs, M=P, callback=count_iterations, tol=tol, restrt=500)
    sol_iter = it_count
    print("Solvant number of iterations: {0}".format(sol_iter))
    soln_vacuum, info = gmres(blocked_vacuum, rhs, M=P_vac, callback=count_iterations, tol=tol, restrt=500)
    print("Vacuum number of iterations (vacuum case): {0}".format(it_count-sol_iter))
    end = time.time()
    solv_time = (end - start) 
    
    soln_fem = soln[:fem_size]
    soln_bem = soln[fem_size:]

    u = dolfinx.fem.Function(fenics_space)
    u.vector.setValues(list(range(0,fem_size)), np.ascontiguousarray(soln_fem))
    dirichlet_data = trace_matrix * soln_fem
    dirichlet_fun = bempp.api.GridFunction(trace_space, coefficients=dirichlet_data)
    neumann_fun = bempp.api.GridFunction(bempp_space, coefficients=soln_bem)
    
    vac_soln_fem = soln_vacuum[:fem_size]
    vac_soln_bem = soln_vacuum[fem_size:]

    u_vac = dolfinx.fem.Function(fenics_space)
    u_vac.vector.setValues(list(range(0,fem_size)), np.ascontiguousarray(vac_soln_fem))
    vac_dirichlet_data = trace_matrix * vac_soln_fem
    vac_dirichlet_fun = bempp.api.GridFunction(trace_space, coefficients=vac_dirichlet_data)
    vac_neumann_fun = bempp.api.GridFunction(bempp_space, coefficients=vac_soln_bem)
    
    ######################### Solvation energy ##################################
    
    bb_tree = dolfinx.geometry.bb_tree(mesh, mesh.topology.dim)
    cells = []
    points_on_proc = []
    # Find cells whose bounding-box collide with the the points
    cell_candidates = dolfinx.geometry.compute_collisions_points(bb_tree, PC)
    # Choose one of the cells that contains the point
    colliding_cells = dolfinx.geometry.compute_colliding_cells(mesh, cell_candidates, PC)
    for i, point in enumerate(PC):
        if len(colliding_cells.links(i))>0:
            points_on_proc.append(point)
            cells.append(colliding_cells.links(i)[0])
    u_values = u.eval(points_on_proc, cells)
    vac_u_values = u_vac.eval(points_on_proc, cells)
    
    # print(u_values)
    
    q_uF = 0
    q_uF_vac = 0
    for i in range(len(PC)):
        Sum =  (u_values[i].real)*Q[i] 
        q_uF = q_uF + Sum
        Sum_vac =  (vac_u_values[i].real)*Q[i] 
        q_uF_vac = q_uF_vac + Sum_vac

    E_Solv = 0.5*4.*np.pi*332.064*q_uF
    # print('Energia de Solvatacion (solvant case): {:7.3f} [kCal/mol]'.format(E_Solv[0]))
    E_Solv_vac = 0.5*4.*np.pi*332.064*q_uF_vac
    # print('Energia de Solvatacion (vacuum case): {:7.3f} [kCal/mol]'.format(E_Solv_vac[0]))
    
    return E_Solv-E_Solv_vac, hmax, it_count, set_time, solv_time

In [3]:
import dolfinx
import dolfinx.io
import dolfinx.geometry
from petsc4py import PETSc
import ufl
from scipy.sparse import csr_matrix

import bempp.api

bempp.api.set_logging_level(level='info')

from bempp.api.external import fenicsx
# from dolfin_utils.meshconvert import meshconvert
import numpy as np
# from bempp.api.external import fenics
# from scipy.sparse.linalg import cg, minres
import pylab as plt
import time

from scipy.sparse.linalg import gmres
from scipy.sparse.linalg import LinearOperator
from bempp.api.assembly.discrete_boundary_operator import InverseSparseDiscreteBoundaryOperator
from bempp.api.assembly.blocked_operator import BlockedDiscreteOperator

from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz
from bempp.api.operators.potential import laplace as lp
 
# Solver data #
tol = 1e-04         # Tolerance
fmm = 1             # Use FMM

# Problem data #
from readpqr_all import readpqr_all

w = 0.10265                                             # kappa
es = 80.                                               # External permitivity (solvant)
em = 1.                                                # Internal permitivity (molecule)

ec = 1.602176e-19 #[C]
kb = 1.380648e-23 #[J/K]
T  = 300 #[K]
S1 = 1 #(ec/(kb*T))  #38.681 [C/J]

# Result collecting lists #
solv_error = []
mesh_size = []
iter_num = []
time_solve = []
time_set = []

# Iteratotion for mesh #
grid_size =  ['1a1t_rna', '1a4t_protein', '1g70_complex', '1g70_rna', '1hji_complex', '1nyb_complex', '1ull_complex', '1zbn_rna']

it_count = 0
def count_iterations(x):
    global it_count
    it_count += 1
    
def epsilon(x,cavities):
    global em,es
    epsilon = em
    if (np.linalg.norm(x-cavities,axis=1)<1e-8).any():
        epsilon = es
    return epsilon

def epsilon_index(x, points):
    return np.where(np.linalg.norm(x-points,axis=1)<1e-8)[0]
       
def fun_exp(x):
    global cavities
    return np.full(x.shape[1],[epsilon(x.transpose()[j],cavities) for j in range(x.shape[1])])   
     
for m in grid_size:
    print(m)
    ######################### Mesh and spaces ##################################
    [PC, Q, R] = readpqr_all("./data-set3/"+m+".pqr")         # Charges, Points of charges and Radiuses
    gauss_points_FEM = np.hsplit(np.loadtxt("./data-set3/"+m+"_Points_FEM_in_cavity.txt"), np.array([3, 4]))
    indices = np.where(gauss_points_FEM[1]>10.)[0]
    cavities = gauss_points_FEM[0][indices,:]
    mesh_file = "./data-set3/"+m+"-converted.xdmf"
    with dolfinx.io.XDMFFile(MPI.COMM_WORLD, mesh_file, "r") as xdmf:
        mesh = xdmf.read_mesh(name="Grid")
        
        it_count = 0
        E_Solv, hmax, num_iter, set_time, solv_time = poisson_boltzman_jn_var(mesh)
        time_set.append(set_time) 
        print("Setup time: {:10.4f}".format(set_time))
        time_solve.append(solv_time)  
        print("Solve time: {:10.4f}".format(solv_time))  
        bempp.api.log(
            "gmres finished in %i iterations and took %.2E sec." % (num_iter, solv_time))
        # print("Number of iterations: {0}".format(it_count))
        iter_num.append(num_iter)
        mesh_size.append(hmax) 
        
        #### Just saving the solution
        print('Energia de Solvatacion : {:7.3f} [kCal/mol]'.format(E_Solv[0]))
        solv_error.append(E_Solv[0])   

1a1t_rna


INFO:bempp:Created grid with id 98f9ef3a-d9b7-4f63-9890-2e5565132d1c. Elements: 118352. Edges: 177528. Vertices: 59178


FEM dofs: 197624
BEM dofs: 59178


INFO:bempp:OpenCL CPU Device set to: pthread-AMD Ryzen Threadripper PRO 3975WX 32-Cores
66 warnings generated.
  prg.build(options_bytes, [devices[i] for i in to_be_built_indices])

66 warnings generated.
  prg.build(options_bytes, [devices[i] for i in to_be_built_indices])

66 warnings generated.
66 warnings generated.
66 warnings generated.
66 warnings generated.
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)



Solvant number of iterations: 159
Vacuum number of iterations (vacuum case): 206


...

Setup time:  1562.9844
Solve time:  6598.8184
Energia de Solvatacion : -7323.466 [kCal/mol]
1zbn_rna


INFO:bempp:Created grid with id 40846a38-d4e8-48a7-a84d-1e7545544fd5. Elements: 164140. Edges: 246210. Vertices: 82072


FEM dofs: 277193
BEM dofs: 82072
Solvant number of iterations: 152
Vacuum number of iterations (vacuum case): 220


INFO:bempp:gmres finished in 372 iterations and took 2.01E+03 sec.


Setup time:   751.3916
Solve time:  2008.7025
Energia de Solvatacion : -9144.831 [kCal/mol]
